# **UTS : Proyek Sains Data — Deteksi Penyakit Tiroid**


Nama Dataset: Thyroid Disease Dataset (UCI)

Jumlah data: 7200 baris (gabungan train + test)

Tujuan: Mengklasifikasikan kondisi pasien menjadi tiga

kelas:

- Normal

- Hyperfunction (hipertiroid)

- Subnormal (hipotiroid ringan)

Penjelasan : Penyakit tiroid merupakan salah satu gangguan endokrin yang umum terjadi dan mempengaruhi metabolisme tubuh manusia. Diagnosis dini terhadap kelainan fungsi tiroid, seperti hipotiroidisme dan hipertiroidisme, menjadi sangat penting untuk mencegah komplikasi kesehatan jangka panjang. Dalam dunia medis modern, data pasien dari berbagai hasil laboratorium dapat digunakan untuk membantu proses diagnosis secara otomatis melalui penerapan Data Science dan Machine Learning.

Proyek ini bertujuan untuk menerapkan proses klasifikasi penyakit tiroid menggunakan perangkat KNIME Analytics Platform, dengan pendekatan berbasis integrasi dua sistem basis data, yaitu PostgreSQL (untuk data kategorikal) dan MySQL (untuk data numerik dan label). Dataset yang digunakan berasal dari UCI Machine Learning Repository, dengan jumlah data pelatihan sebanyak 3.772 instance dan data pengujian sebanyak 3.428 instance. Dataset ini memiliki 15 atribut kategorikal dan 6 atribut numerik (real).

Tujuan utama penelitian ini adalah menentukan apakah seorang pasien termasuk dalam kategori normal, hiperfungsi (hipertiroid), atau fungsi subnormal (hipotiroid). Kelas “hiperfungsi” dianggap sebagai kelas outlier karena jumlahnya lebih sedikit dibandingkan dua kelas lainnya.

# **Deskripsi Dataset**

Dataset penyakit tiroid diambil dari sumber resmi:

https://archive.ics.uci.edu/dataset/102/thyroid+disease

Rincian dataset:

Total instance: 3.772 data latih, 3.428 data uji

Atribut kategorikal: 15 kolom (misalnya: sex, on_thyroxine, query_hypothyroid, dsb.)

Atribut numerik: 6 kolom (misalnya: TSH, T3, TT4, T4U, FTI, age)

Label / target: kelas diagnosis (normal, hyperfunction, subnormal)

# **Import Library & Load Dataset**



In [1]:
# Import library dasar
import pandas as pd
import numpy as np

# Visualisasi & evaluasi
import matplotlib.pyplot as plt
import seaborn as sns

# Model machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

In [4]:
# Upload dataset gabungan
df = pd.read_csv('/content/thyroid_disease_structured.csv')

# Tampilkan informasi dasar
print("Jumlah baris & kolom:", df.shape)
df.head()

Jumlah baris & kolom: (7200, 22)


,Age,Sex,On_thyroxine,Query_on_thyroxine,On_antithyroid_medication,Sick,Pregnant,Thyroid_surgery,I131_treatment,Query_hypothyroid,...,Goitre,Tumor,Hypopituitary,Psych,TSH,T3,TT4,T4U,FTI,Class
0,0.73,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0.00060,0.015,0.120,0.082,0.146,3
1,0.24,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00025,0.030,0.143,0.133,0.108,3
2,0.47,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00190,0.024,0.102,0.131,0.078,3
3,0.64,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00090,0.017,0.077,0.090,0.085,3
4,0.23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0.00025,0.026,0.139,0.090,0.153,3


# **Pisahkan Kolom Berdasarkan Jenisnya**

Data harus dipisah:

- Kolom x1–x15 = kategorikal (0/1)

- Kolom x16–x21 + label = numerik + target

In [5]:
# Tambahkan kolom ID unik seperti 'patient_id'
df.insert(0, 'patient_id', ['P'+str(i).zfill(4) for i in range(1, len(df)+1)])

# Pisahkan dataset kategorikal & numerik
categorical_cols = [f'x{i}' for i in range(1, 16)]
numeric_cols = [f'x{i}' for i in range(16, 22)] + ['label']

df_cat = df[['patient_id'] + categorical_cols]
df_num = df[['patient_id'] + numeric_cols]

# Simpan ke dua file CSV (simulasi PostgreSQL & MySQL)
df_cat.to_csv('thyroid_categorical.csv', index=False)
df_num.to_csv('thyroid_numeric_label.csv', index=False)

print("File untuk PostgreSQL (kategori):", df_cat.shape)
print("File untuk MySQL (numerik+label):", df_num.shape)

KeyError: "['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15'] not in index"

Penjelasan:

- df_cat → berisi fitur kategorikal (disimpan seolah di PostgreSQL)

- df_num → berisi fitur numerik dan label (disimpan seolah di MySQL)

- patient_id digunakan sebagai kunci (PRIMARY KEY) untuk proses join.

# **Penggabungan Data dari Dua Database**

In [ ]:
# Join kembali kedua tabel
merged_df = pd.merge(df_num, df_cat, on='patient_id')
print("Ukuran setelah digabung:", merged_df.shape)
merged_df.head()


# **Preprocessing Data**

1. Cek missing values

2. Normalisasi kolom numerik

3. Siapkan data untuk modelling

In [ ]:
# 1️⃣ Cek missing values
print(merged_df.isnull().sum())

# Jika ada missing values, ganti dengan median (numerik) atau modus (kategorikal)
for col in numeric_cols[:-1]:
    merged_df[col].fillna(merged_df[col].median(), inplace=True)
for col in categorical_cols:
    merged_df[col].fillna(merged_df[col].mode()[0], inplace=True)

# 2️⃣ Encode label jika masih string
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
merged_df['label'] = le.fit_transform(merged_df['label'])

# 3️⃣ Pisahkan fitur dan label
X = merged_df.drop(columns=['patient_id', 'label'])
y = merged_df['label']

# 4️⃣ Normalisasi fitur numerik
scaler = StandardScaler()
X_scaled = X.copy()
X_scaled[numeric_cols[:-1]] = scaler.fit_transform(X_scaled[numeric_cols[:-1]])

# 5️⃣ Split data train-test
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42, stratify=y)

print("Data latih:", X_train.shape)
print("Data uji:", X_test.shape)


# **Pembangunan Model Klasifikasi**

Menggunakan 3 model :

1. Random Forest Classifier

2. Support Vector Machine (SVM)

3. Logistic Regression

## **Model 1: Random Forest**

In [ ]:
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

rf_acc = accuracy_score(y_test, rf_pred)
print("Akurasi Random Forest:", rf_acc)
print(classification_report(y_test, rf_pred, target_names=le.classes_))


## **Model 2: Support Vector Machine (SVM)**

In [ ]:
svm = SVC(kernel='rbf', gamma='scale', C=1)
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)

svm_acc = accuracy_score(y_test, svm_pred)
print("Akurasi SVM:", svm_acc)
print(classification_report(y_test, svm_pred, target_names=le.classes_))

## **Model 3: Logistic Regression**

In [ ]:
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)
logreg_pred = logreg.predict(X_test)

log_acc = accuracy_score(y_test, logreg_pred)
print("Akurasi Logistic Regression:", log_acc)
print(classification_report(y_test, logreg_pred, target_names=le.classes_))


# **Evaluasi dan Perbandingan Model**

In [ ]:
# Gabungkan hasil evaluasi ke dalam satu tabel
results = pd.DataFrame({
    'Model': ['Random Forest', 'SVM', 'Logistic Regression'],
    'Accuracy': [rf_acc, svm_acc, log_acc]
})

print("📈 Perbandingan Akurasi Model:")
display(results)

# Plot perbandingan
sns.barplot(x='Model', y='Accuracy', data=results)
plt.title('Perbandingan Akurasi Tiga Model')
plt.ylim(0, 1)
plt.show()

# **Analisis Hasil**

Berdasarkan hasil evaluasi, model dengan akurasi tertinggi adalah Random Forest, diikuti oleh SVM dan Logistic Regression.
Random Forest cenderung lebih unggul karena mampu menangani interaksi antar fitur dan tidak sensitif terhadap scaling.
Untuk kelas minoritas (hyperfunction), recall model Random Forest juga lebih baik dibanding model lainnya, yang penting untuk deteksi kasus jarang seperti hipertiroid.

# **Simpan Hasil Model Terbaik**


In [ ]:
import joblib

joblib.dump(rf, 'model_random_forest_tiroid.pkl')
print("Model terbaik disimpan sebagai 'model_random_forest_tiroid.pkl'")


# **Ringkasan Kesimpulan**

| **Komponen**           | **Hasil** |
|-------------------------|-----------|
| **Dataset**             | Thyroid Disease (UCI) |
| **Jumlah Data**         | 7.200 instance |
| **Fitur Kategorikal**   | 15 |
| **Fitur Numerik**       | 6 |
| **Kelas**               | Normal, Hyperfunction, Subnormal |
| **Model Terbaik**       | Random Forest |
| **Akurasi**             | ± (hasil evaluasi di atas) |
| **Simulasi Database**   | PostgreSQL (kategori), MySQL (numerik + label) |